# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [28]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

device_id = 1
bb.set_device(device_id)
print(f"BinaryBrain : {bb.get_version_string()}")
print(f"Device : {bb.get_device_name(device_id)}")

BinaryBrain : 4.4.0
Device : NVIDIA GeForce GT 1030


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [29]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut4_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64*2
frame_modulation_size = 31

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [30]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [31]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*4], N=4),
            bb.DifferentiableLut([64*4],   N=4),
            bb.DifferentiableLut([64],     N=4),
            bb.DifferentiableLut([10*4*4], N=4),
            bb.DifferentiableLut([10*4],   N=4),
            bb.AverageLut       ([10],     N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [32]:
#bb.load_networks(data_path, net)
print(net.get_info())

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/469 [00:00<?, ?it/s]

epoch[0] : loss=1.674373 accuracy=0.797000


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[1] : loss=1.673793 accuracy=0.779600


  0%|          | 0/469 [00:00<?, ?it/s]

...

epoch[5] : loss=1.661190 accuracy=0.805400


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[6] : loss=1.663004 accuracy=0.794400


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[7] : loss=1.670652 accuracy=0.795500


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [33]:
# export verilog
bb.load_networks(data_path, net)

os.makedirs(data_path, exist_ok=True)
os.makedirs(rtl_sim_path, exist_ok=True)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut4_simple/MnistLutSimple.v'

In [34]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [35]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x4 の行列が取得できます。

In [36]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [ 96, 287, 745, 140],
       [ 39,  75, 762,  24],
       [730, 179,  33, 337]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [37]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 16)


array([[False, False,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True, False,  True, ..., False, False, False],
       ...,
       [False,  True,  True, ...,  True,  True,  True],
       [ True, False, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [38]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 16)


array([[0.43014205, 0.42491326, 0.49129638, ..., 0.51144713, 0.555975  ,
        0.5427413 ],
       [0.47760394, 0.5795699 , 0.4755156 , ..., 0.50400996, 0.49322742,
        0.5135018 ],
       [0.5573479 , 0.46365649, 0.5132889 , ..., 0.44396335, 0.4907844 ,
        0.52641696],
       ...,
       [0.43358508, 0.507901  , 0.50745225, ..., 0.56991994, 0.5359732 ,
        0.5897219 ],
       [0.5585709 , 0.5086628 , 0.48816174, ..., 0.47554675, 0.44455826,
        0.45831615],
       [0.5047885 , 0.52065736, 0.488165  , ..., 0.47354558, 0.49264285,
        0.48287958]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [39]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 16)


array([[-6.2053790e-05, -2.0684529e-05,  1.6251532e-04, ...,
        -3.0910920e-05,  5.9247246e-05,  1.9749074e-05],
       [ 1.7679745e-04,  1.8322055e-05,  5.8931415e-05, ...,
        -1.9644433e-05, -1.5132249e-05, -6.5481331e-06],
       [ 1.3512690e-05,  1.9456411e-04, -1.6399170e-03, ...,
         1.4646369e-04, -8.7316141e-05,  5.4107513e-04],
       ...,
       [ 3.0486699e-05, -2.2075784e-03,  2.0190361e-03, ...,
        -2.2884172e-04,  2.4977059e-04,  1.1183656e-06],
       [-9.3132257e-10, -1.3969839e-09, -1.3969839e-09, ...,
        -8.7311491e-11, -8.7311491e-11, -7.2759576e-11],
       [ 2.2072345e-07,  7.3254341e-08, -2.3050234e-07, ...,
         8.1527105e-09, -2.5691406e-08, -8.5619831e-09]], dtype=float32)